# Imports

In [9]:
import cv2
from PIL import Image
from sklearn.cluster import BisectingKMeans
import numpy as np
import pandas as pd
import gc
from sklearn.cluster import KMeans
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import glob
from pprint import pprint
import math
from typing import Literal
import xml.etree.ElementTree as ET
import concurrent
import utils

# Globals

In [2]:
NAMESPACES = {
    'pds': 'http://pds.nasa.gov/pds4/pds/v1',
    'isda': 'https://isda.issdc.gov.in/pds4/isda/v1',
}

# Segmentation

### Getting File Paths

In [8]:
# Get directory list

base_dirs = utils.get_paths_by_re('/kaggle/input/isro-chandrayan-ohrc-dataset/other/dataset/','*/dataset/*')
bin_image_paths = []
xml_paths = []    
for base_dir in base_dirs:
    bin_image_paths.extend(utils.get_paths_by_re(base_dir,'data/calibrated/**/*.img'))
    xml_paths.extend(utils.get_paths_by_re(base_dir,'data/calibrated/**/*.xml'))
    
bin_image_paths.sort()
xml_paths.sort()


In [7]:
pd.DataFrame({'image_paths': bin_image_paths, 'xml_paths': xml_paths})

,image_paths,xml_paths
0,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
1,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
2,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
3,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
4,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
5,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
6,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
7,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
8,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...
9,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...,/kaggle/input/isro-chandrayan-ohrc-dataset/oth...


In [ ]:
np.